### Project parameters

In [1]:
## Imports ##
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from keras import layers, optimizers, Model
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.datasets import mnist
from keras.optimizers import Adam
from keras.initializers import RandomUniform
from keras.utils.vis_utils import model_to_dot

from IPython.display import display, SVG

## Constants
num_classes = 10
img_rows, img_cols, img_chan = 28, 28, 1
img_shape = (img_rows, img_cols, img_chan)

metrics = ['accuracy']
verbose = 0

## Data ##
(x_reader, y_reader), (_, _) = mnist.load_data()
x_writer, y_writer = x_reader, y_reader

x_reader = x_reader.reshape(x_reader.shape[0], img_rows * img_cols)
x_reader = x_reader.astype(np.float32) / 255.0

x_writer = (x_writer.astype(np.float32) - 127.5) / 127.5 # Rescale -1 to 1
x_writer = np.expand_dims(x_writer, axis=3)

y_reader = to_categorical(y_reader, num_classes)
y_writer = y_writer.reshape(-1, 1)

history = [[],[],[],[]]

## Hyperparameters ##
# Reading
reader_epochs = 40
reader_validation_split = 0.2
reader_batch_size = 512
reader_kernel_initializer = RandomUniform(minval=0.0000001, 
                                          maxval=0.0001, 
                                          seed=None)
reader_optimizer = optimizers.Adam()
reader_loss = categorical_crossentropy

# Writing
writer_epochs = 22000
writer_latent_dim = 100
writer_batch_size = 32
writer_half_batch = int(writer_batch_size / 2)

writer_optimizer = optimizers.Adam(lr=0.0002,      # lr default:         0.001
                                    beta_1=0.5,     # beta_1 default:     0.9
                                    beta_2=0.999, 
                                    epsilon=None, 
                                    decay=0.0, 
                                    amsgrad=False)
writer_loss = ['binary_crossentropy']

grader_optimizer = writer_optimizer
grader_loss = ['binary_crossentropy', 'sparse_categorical_crossentropy']

school_optimizer = grader_optimizer
school_loss = ['binary_crossentropy','sparse_categorical_crossentropy']

print("Matrix Shapes:")
print("img_shape:\t", img_shape)
print('x_reader:\t', x_reader.shape)
print('y_reader:\t', y_reader.shape)
print('x_writer:\t', x_writer.shape)
print('y_writer:\t', y_writer.shape)

Using TensorFlow backend.


Matrix Shapes:
img_shape:	 (28, 28, 1)
x_reader:	 (60000, 784)
y_reader:	 (60000, 10)
x_writer:	 (60000, 28, 28, 1)
y_writer:	 (60000, 1)


# Models

### Reading model

In [2]:
reader = Sequential()
reader.add(layers.Dense(320, activation='relu', 
                        kernel_initializer=reader_kernel_initializer, 
                        input_shape=[x_reader.shape[1]]))
reader.add(layers.Dropout(0.66))
reader.add(layers.Dense(num_classes, activation='softmax'))
reader.compile(loss=reader_loss, 
               optimizer=reader_optimizer, 
               metrics=metrics)

### Writer models

#### Student

In [3]:
# Input Layers: 2 layers: noise, class_labels
noise = layers.Input(shape=(writer_latent_dim,))
label = layers.Input(shape=(1,), dtype='int32')

# Create an embedding
label_embedding = layers.Embedding(num_classes, writer_latent_dim)(label)
label_embedding = layers.Flatten()(label_embedding)

writer_input = layers.multiply([noise, label_embedding])

# Generator hidden layers
writer_hidden = layers.Dense(128 * 7 * 7, 
                             activation='relu', 
                             input_dim=writer_latent_dim)(writer_input)
writer_hidden = layers.Reshape((7, 7, 128))(writer_hidden)
writer_hidden = layers.BatchNormalization(momentum=0.8)(writer_hidden)
writer_hidden = layers.UpSampling2D()(writer_hidden)
writer_hidden = layers.Conv2D(128, 
                              activation='relu',
                              kernel_size=3, 
                              padding='same')(writer_hidden)
writer_hidden = layers.BatchNormalization(momentum=0.8)(writer_hidden)
writer_hidden = layers.UpSampling2D()(writer_hidden)
writer_hidden = layers.Conv2D(64, 
                              activation='relu', 
                              kernel_size=3, 
                              padding='same')(writer_hidden)
writer_hidden = layers.BatchNormalization(momentum=0.8)(writer_hidden)
writer_image = layers.Conv2D(img_chan, 
                              activation='tanh',
                              kernel_size=3, 
                              padding='same')(writer_hidden)

# Finalize the model
writer = Model([noise, label], writer_image)
writer.compile(loss=writer_loss, 
               optimizer=writer_optimizer)

#### Grader

In [4]:
grader_image = layers.Input(shape=img_shape)

grader_hidden = layers.Conv2D(16, 
                              kernel_size=3, 
                              strides=2,  
                              padding='same', 
                              input_shape=img_shape)(grader_image)
grader_hidden = layers.LeakyReLU(alpha=0.2)(grader_hidden)
grader_hidden = layers.Dropout(0.25)(grader_hidden)
grader_hidden = layers.Conv2D(32, 
                              kernel_size=3, 
                              strides=2, 
                              padding='same')(grader_hidden)
grader_hidden = layers.ZeroPadding2D(padding=((0,1),(0,1)))(grader_hidden)
grader_hidden = layers.LeakyReLU(alpha=0.2)(grader_hidden)
grader_hidden = layers.Dropout(0.25)(grader_hidden)
grader_hidden = layers.BatchNormalization(momentum=0.8)(grader_hidden)
grader_hidden = layers.Conv2D(64, 
                              kernel_size=3, 
                              strides=2, 
                              padding='same')(grader_hidden)
grader_hidden = layers.LeakyReLU(alpha=0.2)(grader_hidden)
grader_hidden = layers.Dropout(0.25)(grader_hidden)
grader_hidden = layers.BatchNormalization(momentum=0.8)(grader_hidden)
grader_hidden = layers.Conv2D(128, 
                              kernel_size=3, 
                              strides=1, 
                              padding='same')(grader_hidden)
grader_hidden = layers.LeakyReLU(alpha=0.2)(grader_hidden)
grader_hidden = layers.Dropout(0.25)(grader_hidden)
grader_hidden = layers.Flatten()(grader_hidden)

grader_valid = layers.Dense(1, 
                            activation='sigmoid')(grader_hidden)
grader_label = layers.Dense(num_classes + 1, 
                            activation='softmax')(grader_hidden)

grader = Model(grader_image, [grader_valid, grader_label])
grader.compile(loss=grader_loss, 
               optimizer=grader_optimizer, 
               metrics=metrics)

# Don't update discriminator during generator training (moving target problem)
grader.trainable = False

# Don't recompile the discriminator so may still be trained independently...
grader_valid, grader_label = grader(writer_image)

#### School house

In [5]:
# Combined model takes generator inputs and has discriminator outputs...
school = Model([noise, label], [grader_valid, grader_label])
school.compile(loss=school_loss, 
               optimizer=school_optimizer)

### Model Details

In [6]:
# SVG(model_to_dot(reader).create(prog='dot', format='svg'))
# SVG(model_to_dot(writer).create(prog='dot', format='svg'))
# SVG(model_to_dot(grader).create(prog='dot', format='svg'))
# SVG(model_to_dot(school).create(prog='dot', format='svg'))

# reader.summary()
# writer.summary()
# grader.summary()
# school.summary()

# Training

In [7]:
reader_hist = reader.fit(x_train, y_train, 
                         epochs=reader_epochs, 
                         batch_size=reader_batch_size, 
                         verbose=verbose, 
                         validation_split=validation_split)

NameError: name 'x_train' is not defined